## Predatory Inclusion in Non-Profit and For-Profit Online Education
### By Christian Michael Smith, Amber D. Villalobos (equal authors), Laura T. Hamilton, and Charlie Eaton

### Code by Christian Smith

### Estimation for Tables 3 and 5

In [ ]:
use data/d_t3_t5_ipeds_scorecard, clear

In [ ]:
describe, size

In [ ]:
set more off

In [ ]:
**********(A) SETUP DATA FOR ANALYSIS**********

*Keep only 4yr institutions
keep if iclevel==1

*Drop observations from the 2020-2021 academic year 
drop if year==2020

*Define sets of control variables
	*Control variable set when dependent variable is student body demographic in nature
global controls_demog "i.year i.control i.hdegofr1 i.locale i.obereg hbcu"
	*Control variable set when dependent variable is a student outcome
global controls_outcome "i.year logenrl blackpct_u latinxpct_u aianpct_u asiapct_u malepct_u upgrntp acceptrate i.control i.hdegofr1 i.locale i.obereg hbcu"

In [ ]:
**********(B) ESTIMATE %ONLINE --> DEPENDENT VARIABLE RELATIONSHIPS**********

*Estimate %online --> student demographics relationships (unconditional)
foreach demogvar in upgrntp blackpct_u {
	foreach control in allcontrols forprofit nonprofit {
		reg `demogvar' pctonline10 if `control'==1, vce(cluster unitid)
		display "Dependent variable:  `demogvar',  Control: `control'"
	}
}


*Estimate %online --> student demographics relationships (conditional on baseline institutional characteristics)
foreach demogvar in upgrntp blackpct_u {
	foreach control in allcontrols forprofit nonprofit {
		reg `demogvar' pctonline10 $controls_demog if `control'==1, vce(cluster unitid)
		display "Dependent variable:  `demogvar',  Control: `control'"
	}
}

*Estimate %online --> student outcome relationships
foreach outcomevar in ret_pcf gradrate6 totloan_a bbrr2_fed_ug_dflt bbrr2_fed_ug_dlnq bbrr2_fed_ug_fbr bbrr2_fed_ug_noprog bbrr2_fed_ug_discharge bbrr2_fed_ug_dfr  bbrr2_fed_ug_makeprog bbrr2_fed_ug_paidinfull {
	foreach control in allcontrols forprofit nonprofit {
		reg `outcomevar' pctonline10 $controls_outcome if `control'==1, vce(cluster unitid)
		display "Dependent variable:  `outcomevar',  Control: `control'"
	}
}

In [ ]:
**********(C) COMPUTE DESCRIPTIVE STATISTICS**********
set line 255
*Describe continuous variables
univar pctonline blackpct_u ret_pcf gradrate6 totloan_a bbrr2_fed_ug_dflt bbrr2_fed_ug_dlnq bbrr2_fed_ug_fbr bbrr2_fed_ug_dfr bbrr2_fed_ug_noprog bbrr2_fed_ug_makeprog bbrr2_fed_ug_paidinfull bbrr2_fed_ug_discharge logenrl latinxpct_u aianpct_u asiapct_u malepct_u upgrntp acceptrate hbcu

*Describe categorical variables
tab year 
tab control 
tab hdegofr1 
tab locale 
tab obereg

*Show online shares by sector and year 
foreach sector in nonprofit forprofit {
	forvalues yr=2012/2019 {
		summ pctonline if year==`yr' & `sector'==1
		display "Sector:  `sector',  Year: `yr'"
	}
}
